In [1]:
import pandas as pd
import numpy as np
from math import sqrt

In [2]:
def distance(user_1, user_2):
    return sqrt(np.sum((user_1 - user_2)**2))     # 유클리드 거리를 계산해주는 함수

In [3]:
def filter_users_without_movie(rating_data, movie_id):
    return rating_data[~np.isnan(rating_data[:,movie_id])]     # movie_id 번째 영화를 평가하지 않은 유저들은 미리 제외해주는 함수

In [4]:
def fill_nan_with_user_mean(rating_data):     # 평점 데이터의 빈값들을 각 유저 평균 값으로 채워주는 함수
    filled_data = np.copy(rating_data)     # 평점 데이터를 훼손하지 않기 위해 복사
    row_mean = np.nanmean(filled_data, axis=0)     # 유저 평균 평점 계산
    
    inds = np.where(np.isnan(filled_data))     # 비어 있는 인덱스들을 구함
    filled_data[inds] = np.take(row_mean, inds[1])     # 빈 인덱스를 유저 평점으로 채운다
    
    return filled_data

In [5]:
def get_k_neighbors(user_id, rating_data, k):     # user_id 에 해당하는 유저의 이웃들을 찾아주는 함수
    distance_data = np.copy(rating_data)     # 평점 데이터를 훼손하지 않기 위해 복사
    distance_data = np.append(distance_data, np.zeros((distance_data.shape[0],1)), axis=1)    # 마지막에 거리 데이터를 담을 열 추가 
    
    for i in range(len(distance_data)):
        row = distance_data[i]
        if i == user_id:     # 같은 유저면 거리를 무한대로 설정
            row[-1] = np.inf
        else:
            row[-1] = distance(distance_data[user_id][:-1], row[:-1])
    
    distance_data = distance_data[np.argsort(distance_data[:,-1])]     # 데이터를 거리 열을 기준으로 정렬
    return distance_data[:k, :-1]     # 가장 가까운 k개의 행만 리턴 ,  마지막(거리) 열은 제외

In [6]:
def predict_user_rating(rating_data, k, user_id, movie_id):     # 예측 행렬에 따라 유저의 영화 평점 예측 값 구하기
    filtered_data = filter_users_without_movie(rating_data, movie_id)    # movie_id 번째 영화를 보지 않은 유저를 데이터에서 제외시킴
    filled_data = fill_nan_with_user_mean(filtered_data)     # 빈 값들이 채워진 새로운 행렬을 만듦
    neighbors = get_k_neighbors(user_id, filled_data, k)     # 유저 user_id 와 비슷한 k개의 유저 데이터를 찾음
    
    return np.mean(neighbors[:,movie_id])
    

In [10]:
RATING_DATA_PATH = 'dataset/ratings.csv'      # 받아올 데이터 경로

np.set_printoptions(precision=2)     # 소수점 둘째 자리까지만 출력

rating_data = pd.read_csv(RATING_DATA_PATH, index_col='user_id').values     # 평점 데이터 불러오기
rating_data_df = pd.DataFrame(rating_data)
rating_data_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,2.0,3.0,4.0,NaN,2.0,3.0,NaN,NaN,NaN,4.0,4.0,NaN,NaN,1.0,NaN,NaN,2.0,5.0,2.0,NaN
1,NaN,NaN,NaN,4.0,NaN,5.0,NaN,NaN,2.0,NaN,4.0,NaN,1.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN
2,2.0,NaN,1.0,NaN,NaN,5.0,5.0,NaN,NaN,NaN,NaN,5.0,3.0,3.0,NaN,3.0,3.0,4.0,NaN,NaN
3,NaN,3.0,5.0,NaN,NaN,NaN,4.0,3.0,NaN,5.0,3.0,NaN,4.0,NaN,NaN,2.0,NaN,NaN,NaN,2.0
4,NaN,NaN,NaN,5.0,NaN,NaN,2.0,2.0,2.0,NaN,3.0,4.0,2.0,5.0,4.0,NaN,NaN,NaN,4.0,2.0
5,NaN,4.0,3.0,NaN,NaN,5.0,NaN,NaN,NaN,3.0,2.0,NaN,NaN,NaN,1.0,NaN,4.0,3.0,NaN,5.0
6,3.0,NaN,NaN,NaN,4.0,5.0,2.0,2.0,NaN,3.0,NaN,4.0,NaN,NaN,2.0,4.0,NaN,NaN,NaN,1.0
7,NaN,5.0,1.0,4.0,NaN,NaN,NaN,3.0,1.0,NaN,1.0,NaN,NaN,4.0,NaN,NaN,2.0,NaN,NaN,4.0
8,NaN,3.0,1.0,1.0,NaN,3.0,NaN,4.0,NaN,1.0,1.0,NaN,4.0,4.0,2.0,NaN,NaN,5.0,3.0,NaN
9,1.0,1.0,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,1.0,NaN,NaN,NaN,4.0,NaN


In [8]:
predict_user_rating(rating_data, 5, 0, 3)     # 5개의 이웃을 써서 유저 0의 영화 3에 대한 예측 평점 구하기 

4.0